In [6]:
def function_1(input_1):
    return input_1 + " First Function "

def function_2(input_2):
    return input_2 + "to Second Function"


In [7]:
from langgraph.graph import Graph

# Define a Langchain graph
workflow = Graph()

workflow.add_node("node_1", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('node_1', 'node_2')

workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [8]:
app.invoke('I am moving from')

'I am moving from First Function to Second Function'

In [9]:
input = 'I am moving from'
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'node_1':
---
I am moving from First Function 

---

Output from node 'node_2':
---
I am moving from First Function to Second Function

---



# Integrating LLM call in the LangGraph

In [10]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview")
llm.invoke("hi")

AIMessage(content='Hi! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_time': 0.028912266, 'prompt_time': 0.003753027, 'queue_time': 0.029514092999999998, 'total_time': 0.032665293}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'stop', 'logprobs': None}, id='run-8c5e948a-ffbc-4bf7-a993-a16108c7771f-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21})

In [15]:
def function_1(input_1):
    complete_query = "Your task is to provide only the topic based on the user query. \
        Only output the topic among: [Japan , Sports]. Don't include reasoning. Following is the user query: " + input_1
    response = llm.invoke(complete_query)
    return response.content

def function_2(input_2):
    TOPIC_UPPER = input_2.upper()
    response = f"Here is the topic in UPPER case: {TOPIC_UPPER}"
    return response

In [16]:
# Define a Langchain graph
workflow = Graph()

workflow.add_node("Agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge('Agent', 'tool')

workflow.set_entry_point("Agent")
workflow.set_finish_point("tool")

app = workflow.compile()

In [17]:
query = "Tell me about Japan's Industrial Growth"
app.invoke(query)

'Here is the topic in UPPER case: JAPAN'

In [18]:
for output in app.stream(query):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'Agent':
---
Japan

---

Output from node 'tool':
---
Here is the topic in UPPER case: JAPAN

---



# RAG Pipeline integration

In [20]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]


In [22]:
###  BGE Embddings

from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

c:\Users\sunny\LangGraph-End-to-End-Course\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\sunny\LangGraph-End-to-End-Course\venv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sunny\.cache\huggingface\hub\models--BAAI--bge-base-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [23]:
### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [24]:
query = "Tell me about llama3?"
docs = retriever.get_relevant_documents(query)
print(docs)

[Document(metadata={'source': '..\\data\\llama3.txt'}, page_content='by Meta AI starting in February 2023.[2][3] The latest version is Llama 3 released in April'), Document(metadata={'source': '..\\data\\llama3.txt'}, page_content='in select regions, and a standalone website. Both services use a Llama 3 model.[12] Reception was'), Document(metadata={'source': '..\\data\\llama3.txt'}, page_content='version of Llama 3 as being "surprisingly capable" given it\'s size.[11]'), Document(metadata={'source': '..\\data\\llama3.txt'}, page_content='Alongside the release of Llama 3, Meta added virtual assistant features to Facebook and WhatsApp in')]


C:\Users\sunny\AppData\Local\Temp\ipykernel_21444\2266497744.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever.get_relevant_documents(query)


In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA

In [26]:
# assign AgentState as an empty dict
AgentState = {}

# messages key will be assigned as an empty array. We will append new messages as we pass along nodes. 
AgentState["messages"] = []

In [27]:
AgentState

{'messages': []}

In [28]:
def function_1(state):
    messages = state['messages']
    question = messages[-1]   ## Fetching the user question
    
    complete_query = "Your task is to provide only the topic based on the user query. \
        Only output the topic among: [Japan , Sports]. Don't include reasoning. Following is the user query: " + question
    response = llm.invoke(complete_query)
    state['messages'].append(response.content) # appending LLM call response to the AgentState
    return state

def function_2(state):
    messages = state['messages']
    question = messages[0] ## Fetching the user question

    template = """Answer the question based only on the following context:
    {context}

    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )
    result = retrieval_chain.invoke(question)
    return result

In [29]:
# Define a Langchain graph
workflow = Graph()

workflow.add_node("Agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge('Agent', 'tool')

workflow.set_entry_point("Agent")
workflow.set_finish_point("tool")

app = workflow.compile()

In [31]:
inputs = {"messages": ["Tell me about llama3 model"]}
app.invoke(inputs)

'The Llama 3 model is a state-of-the-art AI model developed by Meta AI. It was first released in February 2023 and the latest version was released in April. The model has been tested to be capable of performing tasks similar to other top models such as PaLM and Chinchilla. It has been described as "surprisingly capable" given its size. The model is used in various services, including a chatbot and a standalone website, and has received positive reception in select regions.'

In [33]:
inputs = {"messages": ["tell me 3 property of llama3?"]}
app.invoke(inputs)

'1. Llama 3 is "surprisingly capable" given its size.\n2. Llama 3 is the latest version, released in April.\n3. Llama 3 is on par with state of the art models such as PaLM and Chinchilla.'

In [34]:
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'Agent':
---
{'messages': ['tell me 3 property of llama3?', 'Japan', 'Japan']}

---

Output from node 'tool':
---
1. Llama 3 is "surprisingly capable" given its size.
2. It is the latest version of Llama, released in April.
3. It is considered to be one of the state of the art models, such as PaLM and Chinchilla.

---



# Simplyfying the State addition and maintainence

In [35]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [36]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate


class TopicSelectionParser(BaseModel):
    Topic: str = Field(description='Selected Topic')
    #Reasoning: str = Field(description='Reasoning behind topic selection')

parser = PydanticOutputParser(pydantic_object=TopicSelectionParser)

In [49]:
def function_1(state):
    print('-> Calling Agent ->')
    messages = state['messages']
    question = messages[-1]   ## Fetching the user question
    
    templete = """ Your task is to provide only the topic based on the user query. 
        Only output the topic among: [Japan , Sports , Not Related]. Don't include reasoning. Following is the user query:  {question}
        {format_instructions} """
    prompt = PromptTemplate(template=templete,
                                    input_variables=[question],
                                    partial_variables={
                                        "format_instructions" : parser.get_format_instructions()                                    }
                                    )
    chain = prompt | llm | parser

    response = chain.invoke({"question":question,"format_instructions" : parser.get_format_instructions() })

    print(response)

    return {"messages": [response.Topic]}

def function_2(state):
    print('-> Calling RAG ->')
    messages = state['messages']
    question = messages[0] ## Fetching the user question

    template = """Answer the question based only on the following context:
    {context}

    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )
    result = retrieval_chain.invoke(question)
    return  {"messages": [result]}

def function_3(state):
    print('-> Calling LLM ->')

    messages = state['messages']
    question = messages[0] ## Fetching the user question

    # Normal LLM call
    complete_query = "Anwer the follow question with you knowledge of the real world. Following is the user question: " + question
    response = llm.invoke(complete_query)
    return {"messages": [response.content]}


def router(state):
    print('-> Router ->')
    
    messages = state["messages"]
    last_message = messages[-1]
    print(last_message)
    if 'Japan' in last_message or 'Sports' in last_message:
        return 'RAG Call'
    else:
        return 'LLM Call'

In [50]:
from langgraph.graph import StateGraph,END

graph = StateGraph(AgentState) ### StateGraph with AgentState

graph.add_node("agent", function_1)
graph.add_node("RAG", function_2)
graph.add_node("LLM", function_3)

graph.set_entry_point("agent")

In [51]:
# conditional edges are controlled by our router
graph.add_conditional_edges(
    "agent",  # where in graph to start
    router,  # function to determine which node is called
    {
        'RAG Call': "RAG",
        'LLM Call': "LLM",
    }
)

In [52]:
graph.add_edge("RAG", END)
graph.add_edge("LLM", END)

app = graph.compile()

In [53]:
inputs = {"messages": ["Tell me about Japan's Industrial Growth"]}
out = app.invoke(inputs)

-> Calling Agent ->
Topic='Japan'
-> Router ->
Japan
-> Calling RAG ->


In [54]:
out['messages']

["Tell me about Japan's Industrial Growth",
 'Japan',
 "There is no information about Japan's Industrial Growth in the provided context."]

In [55]:
out['messages'][-1]

"There is no information about Japan's Industrial Growth in the provided context."

In [56]:
inputs = {"messages": ["Who came fourth for Ireland at the outdoor European Running Championships in 1998?"]}
out = app.invoke(inputs)
print(out)

-> Calling Agent ->
Topic='Sports'
-> Router ->
Sports
-> Calling RAG ->
{'messages': ['Who came fourth for Ireland at the outdoor European Running Championships in 1998?', 'Sports', "I'm sorry but I do not have the answer to that question."]}


In [57]:
inputs = {"messages": ["Tell me about Bert Model"]}
out = app.invoke(inputs)

-> Calling Agent ->
Topic='Not Related'
-> Router ->
Not Related
-> Calling LLM ->


In [58]:
print(out['messages'])

['Tell me about Bert Model', 'Not Related', "BERT (Bidirectional Encoder Representations from Transformers) is a pre-trained language model developed by Google in 2018. It's designed to understand natural language processing (NLP) tasks and has been fine-tuned to perform well on a wide range of NLP tasks such as question answering, sentiment analysis, and text classification.\n\nBERT uses a multi-layer bidirectional transformer encoder to generate contextualized representations of words in the input sequence. These representations can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of NLP tasks.\n\nBERT has several key features that make it effective:\n\n1. **Contextual Understanding**: BERT is trained to understand the context in which a word is used, which is crucial for accurately modeling language. It does this by looking at the entire input sequence, rather than just the immediate context of a word.\n\n2. **Pre-training**: BER

In [59]:
print(out['messages'][-1])

BERT (Bidirectional Encoder Representations from Transformers) is a pre-trained language model developed by Google in 2018. It's designed to understand natural language processing (NLP) tasks and has been fine-tuned to perform well on a wide range of NLP tasks such as question answering, sentiment analysis, and text classification.

BERT uses a multi-layer bidirectional transformer encoder to generate contextualized representations of words in the input sequence. These representations can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of NLP tasks.

BERT has several key features that make it effective:

1. **Contextual Understanding**: BERT is trained to understand the context in which a word is used, which is crucial for accurately modeling language. It does this by looking at the entire input sequence, rather than just the immediate context of a word.

2. **Pre-training**: BERT is pre-trained on a large corpus of text, such as t

# Adding tools

In [60]:
import json
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

@tool
def multiply(first_number: int, second_number: int):
    """Multiplies two numbers together."""
    return first_number * second_number

In [61]:
model_with_tools = llm.bind(tools=[convert_to_openai_tool(multiply)])

In [62]:
response = model_with_tools.invoke('What is 35 * 46?')

In [63]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kh86', 'function': {'arguments': '{"first_number": 35, "second_number": 46}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 209, 'total_tokens': 242, 'completion_time': 0.102648621, 'prompt_time': 0.016858735, 'queue_time': 0.005984724, 'total_time': 0.119507356}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-93c696b1-af2b-4532-aa99-79f59934755b-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 35, 'second_number': 46}, 'id': 'call_kh86', 'type': 'tool_call'}], usage_metadata={'input_tokens': 209, 'output_tokens': 33, 'total_tokens': 242})

In [64]:
tool_calls = response.additional_kwargs.get('tool_calls')
tool_calls

[{'id': 'call_kh86',
  'function': {'arguments': '{"first_number": 35, "second_number": 46}',
   'name': 'multiply'},
  'type': 'function'}]

In [65]:
type(tool_calls)

list

In [66]:
for tool_call in tool_calls:
    print('Function Name:',tool_call.get('function').get('name'))
    print('Function Arguments:',tool_call.get('function').get('arguments'))
    print(tool_call)

Function Name: multiply
Function Arguments: {"first_number": 35, "second_number": 46}
{'id': 'call_kh86', 'function': {'arguments': '{"first_number": 35, "second_number": 46}', 'name': 'multiply'}, 'type': 'function'}


In [67]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [68]:
from langgraph.graph import StateGraph,END

graph = StateGraph(AgentState) ### StateGraph with AgentState


def invoke_model(state):
    messages = state['messages']
    question = messages[-1]   ## Fetching the user question
    return {"messages":[model_with_tools.invoke(question)]}


graph.add_node("agent", invoke_model)

def invoke_tool(state):
    tool_calls = state['messages'][-1].additional_kwargs.get("tool_calls", [])
    multiply_call = None

    for tool_call in tool_calls:
        if tool_call.get("function").get("name") == "multiply":
            multiply_call = tool_call

    if multiply_call is None:
        raise Exception("No adder input found.")

    res = multiply.invoke(
        json.loads(multiply_call.get("function").get("arguments"))
    )

    return {"messages" : [res]
    }

graph.add_node("tool", invoke_tool)

graph.add_edge("tool", END)

graph.set_entry_point("agent")

In [69]:
def router(state):
    tool_calls = state['messages'][-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "end"

graph.add_conditional_edges("agent", router, {
    "multiply": "tool",
    "end": END,
})

In [70]:
app = graph.compile()

output = app.invoke({"messages": ["What is 123 * 456?"]})

In [71]:
output

{'messages': ['What is 123 * 456?',
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_j9nd', 'function': {'arguments': '{"first_number": 123, "second_number": 456}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 209, 'total_tokens': 242, 'completion_time': 0.10267339, 'prompt_time': 0.016128917, 'queue_time': 0.006846070999999999, 'total_time': 0.118802307}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2272cb4b-07d0-44c6-a9b8-e4a82fed8ecf-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 123, 'second_number': 456}, 'id': 'call_j9nd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 209, 'output_tokens': 33, 'total_tokens': 242}),
  56088]}

In [72]:
output['messages'][-1]

56088

In [73]:
output = app.invoke({"messages": ["What is LLM?"]})

In [74]:
output

{'messages': ['What is LLM?',
  AIMessage(content='LLM can stand for several things, depending on the context:\n\n1. Large Language Model: LLM is a term used in the field of artificial intelligence, particularly in natural language processing. It refers to a language model that is trained on a large dataset and has a large number of parameters, allowing it to understand and generate complex sentences and paragraphs.\n\n2. Master of Laws: LLM is also an abbreviation for the Master of Laws degree, which is a postgraduate law degree that is designed for students who have already completed a first degree in law or a related field. It is typically a one-year full-time or two-year part-time program that allows students to specialize in a particular area of law.\n\n3. LLM can also stand for other things, such as "Live Life to the Max" or "Love Life More," but these are less common uses of the abbreviation.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 178, 'p

In [75]:
print(output['messages'][-1])

content='LLM can stand for several things, depending on the context:\n\n1. Large Language Model: LLM is a term used in the field of artificial intelligence, particularly in natural language processing. It refers to a language model that is trained on a large dataset and has a large number of parameters, allowing it to understand and generate complex sentences and paragraphs.\n\n2. Master of Laws: LLM is also an abbreviation for the Master of Laws degree, which is a postgraduate law degree that is designed for students who have already completed a first degree in law or a related field. It is typically a one-year full-time or two-year part-time program that allows students to specialize in a particular area of law.\n\n3. LLM can also stand for other things, such as "Live Life to the Max" or "Love Life More," but these are less common uses of the abbreviation.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 178, 'prompt_tokens': 206, 'total_tokens': 384, 'co